In [1]:
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = [go.Bar(
            x=['giraffes', 'orangutans', 'monkeys'],
            y=[20, 14, 23]
    )]

#py.iplot(data, filename='basic-bar')

In [2]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('postgresql://read_only_user@datathon.data-lab.io:5432/postgres')
import numpy as np
from sklearn import preprocessing

/user/ivo.brydl/.local/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [ ]:
sql

In [27]:
import warnings
warnings.filterwarnings('ignore')

#words = ['flüchtlinge', 'facebook']
words = ['flüchtlinge', 'gdpr','facebook', 'altern','alternative','fakten','bürger','bürgerliche','werte','zivilgesellschaft','gesellschaft','gemeinschaft','kritisches','denken ','zivilgesellschaftliche','organisation','kultur','demokratie','spenden','bildung','wahlen','engagement','europäische','union','fake', 'news','gefälschte','nachrichten','feminismus','ausländer','pressefreiheit','geschlechterungleichheit','ermächtigung','menschenrechte','identität','antiliberal','medien','nichtregierungsorganisation','non-profit-organisation ','partizipation','teilhabe','beiteiligung','populismus','armut','öffentlichkeit','soziale','innovation ','soziale','interaktion','soziale','bewegungen','willensbildung','integration','integration','vielfalt','vernetzung','ehrenamtliche','freiwillige','ehrenamtliches','freiwilliges','engagement','digitalisierung','datenschutz','minderheit','toleranz','flüchtlinge','migration','kooperation','zusammenarbeit','nachhaltigkeit','gesundheit','öffentliches','gesundheitswesen','wohlbefinden','geschlechtergleichheit','energie','innovation','umwelt ','umweltschutz','frieden','gerechtigkeit','hunger','unterernährung','diskriminierung','soziale','benachteiligung','hassverbrechen ','globalisierung','ethik','einkommen','respekt','solidarität','nachhaltige','entwicklung','fairer','handel','kinder','jugend','zukunft','gesellschaft ','bevölkerung','einwohner','öffentliches','engagement','bürgerbeteiligung','deliberative','demokratie','ziviler','dialog','staatsbürgerschaft','urbanisierung','flucht','asyl','asylwerber','humanitäre','hilfe','krisengebiet']
#words = ['altern','alternative','fakten','bürger','bürgerliche','werte','zivilgesellschaft']
#words = ['feminismus', 'flüchtlinge', 'facebook']
#words = ['werte']

#words = ['flüchtling']
words = ['fake']

#words=['integration | zuwanderung | gesellschaftliche | globalisierung | normen | ethik | armut | sozialer | krise | ökonomische']



words = list(set(words))
l = len(words)
i=0
traces = []
for word in words:
    sql = "select * from public.standard where document @@ to_tsquery(\'" + word + "\');"
    #sql = "select * from public.standard where document @@ to_tsquery(\'integration | zuwanderung | gesellschaftliche | globalisierung | normen | ethik | armut | sozialer | krise | ökonomische \');"
    #print(sql)
    tmp = pd.read_sql_query(sql,con=engine)
    
    tmp['ym'] = pd.to_datetime(tmp['pubdate']).map(lambda dt: dt.replace(day=1))
    tmp["cont"] = tmp.document.str.contains(word)
    tmp = tmp[tmp["cont"]==True]
    tmpg = tmp.groupby(tmp['ym']).count()
    tmpg["cnt"] = tmpg["id"]
    tmpg = tmpg.drop(["id", "title", "description", "pubdate", "link", "document", "cont"], axis=1)
    if (tmpg.size>0 and tmpg.cnt.sum()>50 ):
        min_max_scaler = preprocessing.MinMaxScaler()
        tmpg["cnt_norm"] = min_max_scaler.fit_transform(tmpg["cnt"].reshape(-1,1))

        breaky = '2017-08-01'
        hist = tmpg.ix[:breaky].cnt_norm.mean()
        recent = tmpg.ix[breaky:].cnt_norm.mean()
        if (recent/hist>1.5 or 1==1) :
            traces.append(go.Scatter(
                x=tmpg.index,
                y=tmpg.cnt_norm,
                name=word,
                mode='lines',
                connectgaps=True,
            ))
    i+=1


fig = go.Figure(data=traces)
py.iplot(fig, filename='words-in-time')